In [7]:
import pandas as pd
from sqlalchemy import create_engine, types

In [8]:
from collections import defaultdict
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Autenticación
auth_manager = SpotifyClientCredentials(client_id="ce08bdf0cb494c72889aba4fe16050ba", client_secret="06507eb95cff48629e2db1eea2395f88")
sp = spotipy.Spotify(auth_manager=auth_manager)

In [17]:

def extraer_datos_por_genero_y_rango(genero, rango_anyo, resultados_por_anyo=200):
    albumes = defaultdict(list)
    canciones = defaultdict(list)

    for o in range(0, 150, 50):  # Buscar hasta 200 artistas
    
        artistas = sp.search(q=f'genre:{genero}', type='artist', limit=50, offset=o)
        for artista in artistas['artists']['items']:
            artista_id = artista['id']
            artista_nombre = artista['name']
            print(len(artista_nombre))

            # Extraer álbumes del artista
            albumes_data = sp.artist_albums(artista_id, album_type=['album', 'single'], limit=50)
            for album in albumes_data['items']:
                try:
                    album_lanzamiento = int(album['release_date'][:4])
                    if album_lanzamiento in rango_anyo:
                        albumes[album_lanzamiento].append({
                            'artista': artista_nombre,
                            'genero': genero,
                            'tipo': album['album_type'],
                            'nombre': album['name'],
                            'año': album_lanzamiento,
                            'id': album['id']
                        })
                except (KeyError, ValueError, TypeError):
                    continue
            
            # Extraer top tracks del artista (sin vincular a álbumes)
            try:
                top_tracks = sp.artist_top_tracks(artista_id)
                for track in top_tracks['tracks']:
                    try:
                        track_lanzamiento = int(track['album']['release_date'][:4])
                        if track_lanzamiento in rango_anyo:
                            canciones[track_lanzamiento].append({
                                'artista': artista_nombre,
                                'genero': genero,
                                'tipo': 'cancion',
                                'nombre': track['name'],
                                'año': track_lanzamiento,
                                'id': track['id']
                            })

                    except (KeyError, ValueError, TypeError):
                        continue
            except:
                continue

            # Parar si ya tenemos suficientes resultados para todos los años
            if all(len(albumes.get(anyo, [])) >= resultados_por_anyo and len(canciones.get(anyo, [])) >= resultados_por_anyo for anyo in rango_anyo):
                break

    # Devolver resultados
    return [
        {'tipo': 'album', **data} for anyo in rango_anyo for data in albumes[anyo][:resultados_por_anyo]
    ] + [
        {'tipo': 'cancion', **data} for anyo in rango_anyo for data in canciones[anyo][:resultados_por_anyo]
    ]



In [18]:
tastyinfo_2018_2022 = extraer_datos_por_genero_y_rango('rock', range(2018, 2023))

13
7
12
13
10
7
13
6
13
10
9
11
9
9
11
4
15
5
8
6
9
19
11
15
7
15
28
10
11
13
12
13
5
13
18
11
12
2
9
12
14
17
6
10
21
14
15
5
8
4
7
16
13
5
4
14
9
14
9
13
14
10
8
11
7
11
12
5
8
7
17
7
11
13
19
9
6
21
8
24
7
9
10
12
16
10
4
10
12
10
12
11
12
23
13
12
5
13
4
17
11
4
7
11
7
6
12
15
12
9
9
14
11
13
6
6
23
7
24
7
17
10
17
15
6
17
10
17
14
11
14
6
10
24
8
4
14
13
15
14
11
15
13
11
19
9
17
3
5
8


In [11]:
type(tastyinfo_2018_2022)

list

In [12]:
tastyinfo_2018_2022 

[{'tipo': 'album',
  'artista': 'Bunbury',
  'genero': 'rock',
  'nombre': 'Canciones 1987-2017 (2018 Remaster)',
  'año': 2018,
  'id': '6z14BU1sSs5vpswjtNRz5t'},
 {'tipo': 'album',
  'artista': 'R.E.M.',
  'genero': 'rock',
  'nombre': 'R.E.M. At The BBC (Live)',
  'año': 2018,
  'id': '35B6Tl6RKNSP6fyZrzR6Qb'},
 {'tipo': 'album',
  'artista': 'David Bowie',
  'genero': 'rock',
  'nombre': 'Glastonbury 2000 (Live)',
  'año': 2018,
  'id': '6frjpRu2P0fdF9xSS9stjY'},
 {'tipo': 'album',
  'artista': 'David Bowie',
  'genero': 'rock',
  'nombre': "Welcome To The Blackout (Live London '78)",
  'año': 2018,
  'id': '0blyxTUATvmwqDcVM0dGt6'},
 {'tipo': 'album',
  'artista': 'Muse',
  'genero': 'rock',
  'nombre': 'Simulation Theory (Super Deluxe)',
  'año': 2018,
  'id': '5OZgDtx180ZZPMpm36J2zC'},
 {'tipo': 'album',
  'artista': 'Andrés Calamaro',
  'genero': 'rock',
  'nombre': 'Cargar La Suerte',
  'año': 2018,
  'id': '5y3l6sn4M3ACns2H51wGrl'},
 {'tipo': 'album',
  'artista': 'Bon Jovi',

In [18]:
df_2018_2022_sp = pd.DataFrame(tastyinfo_2018_2022)
print(df_2018_2022_sp)

        tipo               artista genero  \
0      album               Bunbury   rock   
1      album                R.E.M.   rock   
2      album                  Muse   rock   
3      album           David Bowie   rock   
4      album           David Bowie   rock   
..       ...                   ...    ...   
669  cancion              Iggy Pop   rock   
670  cancion                  LiSA   rock   
671  cancion                  LiSA   rock   
672  cancion                  LiSA   rock   
673  cancion  Amistades Peligrosas   rock   

                                        nombre   año                      id  
0          Canciones 1987-2017 (2018 Remaster)  2018  6z14BU1sSs5vpswjtNRz5t  
1                     R.E.M. At The BBC (Live)  2018  35B6Tl6RKNSP6fyZrzR6Qb  
2             Simulation Theory (Super Deluxe)  2018  5OZgDtx180ZZPMpm36J2zC  
3                      Glastonbury 2000 (Live)  2018  6frjpRu2P0fdF9xSS9stjY  
4    Welcome To The Blackout (Live London '78)  2018  0blyxTUATv

In [ ]:
df_2018_2022_sp.to_json(
    'rock-extraccion-pruebas-json.json',  # Nombre del archivo de salida
    orient='records',       # Formato legible (lista de registros)
    force_ascii=False,      # Permite caracteres especiales (ñ, á, é, etc.)
    indent=4                # Formato indentado para legibilidad
)